# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
from holoviews import opts

# Import API key
from api_keys import geoapify_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,24.75,88,75,2.68,EC,1699248190
1,1,onega,63.9061,38.1404,1.72,99,100,2.26,RU,1699248190
2,2,atafu village,-8.5421,-172.5159,29.07,71,100,1.47,TK,1699248191
3,3,suva,-18.1416,178.4415,29.71,70,40,3.60,FJ,1699248191
4,4,tiksi,71.6872,128.8694,-24.60,99,43,2.31,RU,1699248191


In [10]:
print(city_data_df["Humidity"])

0      88
1      99
2      71
3      70
4      99
       ..
569    87
570    63
571    76
572    77
573    17
Name: Humidity, Length: 574, dtype: int64


In [11]:
non_neg_humid = city_data_df[city_data_df['Humidity'] > 0]
non_neg_humid

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,24.75,88,75,2.68,EC,1699248190
1,1,onega,63.9061,38.1404,1.72,99,100,2.26,RU,1699248190
2,2,atafu village,-8.5421,-172.5159,29.07,71,100,1.47,TK,1699248191
3,3,suva,-18.1416,178.4415,29.71,70,40,3.60,FJ,1699248191
4,4,tiksi,71.6872,128.8694,-24.60,99,43,2.31,RU,1699248191
...,...,...,...,...,...,...,...,...,...,...
569,569,alegrete,-29.7831,-55.7919,12.92,87,16,4.00,BR,1699248265
570,570,qarazhal,48.0078,70.7903,6.47,63,55,5.39,KZ,1699248265
571,571,kavieng,-2.5744,150.7967,28.47,76,100,3.11,PG,1699248266
572,572,nouadhibou,20.9310,-17.0347,19.99,77,30,8.23,MR,1699248266


In [12]:
city_data_df = city_data_df.dropna()
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,24.75,88,75,2.68,EC,1699248190
1,1,onega,63.9061,38.1404,1.72,99,100,2.26,RU,1699248190
2,2,atafu village,-8.5421,-172.5159,29.07,71,100,1.47,TK,1699248191
3,3,suva,-18.1416,178.4415,29.71,70,40,3.60,FJ,1699248191
4,4,tiksi,71.6872,128.8694,-24.60,99,43,2.31,RU,1699248191
...,...,...,...,...,...,...,...,...,...,...
569,569,alegrete,-29.7831,-55.7919,12.92,87,16,4.00,BR,1699248265
570,570,qarazhal,48.0078,70.7903,6.47,63,55,5.39,KZ,1699248265
571,571,kavieng,-2.5744,150.7967,28.47,76,100,3.11,PG,1699248266
572,572,nouadhibou,20.9310,-17.0347,19.99,77,30,8.23,MR,1699248266


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
#city_data_df = city_data_df.dropna()
humidity = city_data_df["Humidity"].astype(int)
humidity_mahp = non_neg_humid.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_height=850,
    frame_width=850,
    size = "Humidity",
    scale = 2,
    hover_cols = ["City", "Country"],
    color = "City"
    #ValueError [Call holoviews.ipython.show_traceback() for details] Screen sizes must be positive
    #why the error? no humidity values are negative...
    #long and lat were flipped causing a negative error
)
humidity_mahp

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
# i want beaches, so 90F is about 32ish in C and 80F is about 26C
beaches_df = city_data_df.loc[(city_data_df["Max Temp"]>26)&(city_data_df["Max Temp"]<33)]
#missplaced a parenthesis
#what is a good windsped for beach?
#i want waves, 8mph-10mph, lowered speed since they all get removed
beaches_df = beaches_df.loc[(beaches_df["Wind Speed"]>8)&(beaches_df["Wind Speed"]<10)]
#since we have good temps and breezes, we just need some shade
#beaches_df = beaches_df.loc[(beaches_df["Cloudiness"]<35)]
#shade is too limiting
# Drop any rows with null values
beaches_df = beaches_df.dropna()
# Display sample data
beaches_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,west island,-12.1568,96.8225,28.99,74,40,8.23,CC,1699248194
45,45,utrik,11.2278,169.8474,28.13,80,90,8.15,MH,1699248196
204,204,enewetak,11.3474,162.3373,28.44,72,95,8.31,MH,1699248219
303,303,mejit,10.2753,170.8646,28.06,74,64,9.46,MH,1699248230
344,344,mata-utu,-13.2816,-176.1745,28.01,80,99,9.79,WF,1699248234
508,508,rongelap,11.1635,166.8941,28.22,76,97,8.85,MH,1699248203


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = beaches_df.copy()
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(10, "Hotel Name", "")
hotel_df = hotel_df.reset_index()
hotel_df = hotel_df.drop(columns=["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date", "index"])
#^include dropping index to remove it after resetting^
# Display sample data
#we need to drop some columns
hotel_df
#are we supposed to have an empty dataframe?

,City,Lat,Lng,Humidity,Country,Hotel Name
0,west island,-12.1568,96.8225,74,CC,
1,utrik,11.2278,169.8474,80,MH,
2,enewetak,11.3474,162.3373,72,MH,
3,mejit,10.2753,170.8646,74,MH,
4,mata-utu,-13.2816,-176.1745,80,WF,
5,rongelap,11.1635,166.8941,76,MH,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

https://apidocs.geoapify.com/docs/places/#about

In [28]:
# Set parameters to search for a hotel
#they want 10000 meters
radius = 10000
params_h = {
   "categories": "accommodation.hotel",
    "apiKey": geoapify_key
    #^^this apikey needs to be there, if not then we cannot find hotels^^
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame #so we dont need a blank df?
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lats = row["Lat"]
    longs = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params_h["filter"] = f"circle:{longs},{lats},{radius}"
    #^this one include radius^^
    params_h["bias"] = f"proximity:{longs},{lats}"
    #^^this one sets the starting point?^^
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params_h)
    
    # Convert the API response to JSON format
    name_add_json = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_add_json["features"][0]["properties"]["name"]
        #TypeError: 'Response' object is not subscriptable
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df # i think we need an empty datafrae to begin with...
#nevermind, it gets added on to the same row!(if found)

Starting hotel search
west island - nearest hotel: Cocos Village Bungalows
utrik - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
mejit - nearest hotel: No hotel found
mata-utu - nearest hotel: Gîte Oceania
rongelap - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Country,Hotel Name
0,west island,-12.1568,96.8225,74,CC,Cocos Village Bungalows
1,utrik,11.2278,169.8474,80,MH,No hotel found
2,enewetak,11.3474,162.3373,72,MH,No hotel found
3,mejit,10.2753,170.8646,74,MH,No hotel found
4,mata-utu,-13.2816,-176.1745,80,WF,Gîte Oceania
5,rongelap,11.1635,166.8941,76,MH,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
#copy and paste from above
%%capture --no-display

# Configure the map plot
#city_data_df = city_data_df.dropna()
humidity_two = hotel_df["Humidity"].astype(int)
hotel_mahp = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_height=850,
    frame_width=850,
    size = "Humidity",
    scale = 2,
    hover_cols = ["City", "Country"],
    color = "City"
)
# Display the map
hotel_mahp

UsageError: Line magic function `%%capture` not found.
